# Downgrade and Dictionary Attack Against WPA3 and Timing Attack

In [ ]:
# hostapd-wpe.conf
interface=wlan0
ssid=TestWPA3
channel=6
wpa=2
wpa_key_mgmt=SAE
wpa_pairwise=CCMP
rsn_pairwise=CCMP
ieee80211w=2
sae_password=testpassword


**interface** = wlan0<br>
**Purpose**: Specifies the wireless network interface to be used.<br>
**Reason**: wlan0 is typically the default wireless interface name on many systems. This is necessary to identify which network interface will broadcast the rogue AP. <br>

**ssid** = TestWPA3<br>
**Purpose**: Defines the SSID (Service Set Identifier) for the rogue AP.<Br>
**Reason**: The SSID is the network name that clients will see. Naming it TestWPA3 indicates that this is a WPA3 network, encouraging clients to connect for testing purposes.<br><Br>
**channel**=6<br>
**Purpose**: Sets the channel on which the rogue AP will operate.
**Reason**: Channel 6 is a common channel for Wi-Fi networks. Choosing a commonly used channel increases the likelihood of capturing client handshakes, especially in busy environments.<Br><Br>
**wpa**=2:<br>
**Purpose**: Enables WPA2/WPA3 security on the AP.<Br>
**Reason**: WPA3 is based on the WPA2 standard, so this setting ensures the AP supports WPA3 while maintaining compatibility with WPA2.<Br><Br>
**wpa_key_mgmt**=SAE:<br>
**Purpose**: Specifies the key management protocol as SAE (Simultaneous Authentication of Equals).<br>
**Reason**: SAE is the key exchange protocol used by WPA3, replacing the PSK (Pre-Shared Key) method used by WPA2. It's the primary target in a DragonBlood attack.<br><br>
**wpa_pairwise**=CCMP and rsn_pairwise=CCMP:<br>
**Purpose**: Sets the encryption method to CCMP (Counter Mode with Cipher Block Chaining Message Authentication Code Protocol).<br>
Reason: CCMP is the encryption standard used in WPA3, offering strong security. It's essential to use the correct encryption to accurately mimic a WPA3 environment.<br><br>
**ieee80211w**=2:<br>
**Purpose**: Enables Management Frame Protection (MFP), which is mandatory in WPA3.<br>
**Reason**: MFP is a crucial feature of WPA3, providing protection against certain types of attacks. Setting it to 2 ensures it's required, aligning with WPA3 standards.<br><br>
**sae_password**=testpassword:<br>
**Purpose**: Sets the password for the SAE handshake.<Br>
**Reason**: This is the password that clients need to connect to the network. In a testing environment, a simple password is chosen to make it easier to capture and analyze handshakes.

In [ ]:
#Launch the Rouge AP
sudo hostapd-wpe hostapd-wpe.conf

Purpose of Launching the Rogue AP (Access Point) in the context of a DragonBlood attack is to create a malicious Wi-Fi network that mimics a legitimate one. This fake network is used to Capture Handshake Data When a client device tries to connect to the Rogue AP, it will go through the WPA3 handshake process. By setting up the Rogue AP with WPA3 configuration (like SAE), an attacker can capture the handshake data for analysis and exploitation.
The Rogue AP can be configured to intentionally support weaker security protocols or exploit downgrade vulnerabilities. For example, it might trick a client into using WPA2 instead of WPA3, making it easier to capture and crack the handshake. Once a client connects to the Rogue AP, the attacker can intercept and potentially decrypt the traffic between the client and the AP. This can lead to the extraction of sensitive information such as passwords or other credentials.

To capture traffic with Wireshark during a DragonBlood attack, then select the appropriate capture filter, like `ether proto 0x888e`, to focus specifically on EAPOL packets, which are part of the WPA3 handshake. Start capturing traffic by clicking the blue shark fin icon, and then connect a client device to the Rogue AP you’ve set up. As traffic is captured, monitor the interface for EAPOL packets, which indicate the handshake process.

In [ ]:
# Use hcxdumptool to capture handshake
sudo hcxdumptool -i wlan0 -o capture.pcapng --enable_status=1


Performing the downgrade attack would be the next stage. This command uses hcxdumptool, a tool designed to capture WPA handshakes, request PMKID (Pairwise Master Key Identifier), and perform several other Wi-Fi-related attacks.

-i wlan0 specifies the interface to use for capturing traffic.<br>
-o capture.pcapng tells the tool to save the captured packets to a file named capture.pcapng.<br>
--enable_status=1 provides real-time status updates in the terminal, helping you monitor the progress of the capture.<Br><br>
In a downgrade attack, the goal is to force a client that is supposed to connect using the more secure WPA3 protocol to instead connect using the older, less secure WPA2 protocol. By capturing the handshake when a client connects using WPA2, an attacker can attempt to crack the WPA2 handshake using traditional methods, which are less resistant to attack compared to WPA3's SAE.

In [ ]:
# Convert to hashcat format
sudo hcxpcapngtool -o hashcat.hc22000 capture.pcapng

This command converts captured WPA handshakes from the .pcapng format into a format that is compatible with hashcat to crack the password.
-o hashcat.hc22000 specifies the output file in a format that hashcat understands, using mode 22000, which covers both WPA-EAPOL and WPA-PMKID attacks.<br>

After forcing the client to connect via WPA2 (through the downgrade attack), the captured handshake can be converted and fed into hashcat for a brute-force attack. This is possible because WPA2, especially with weak passwords, is more vulnerable to such attacks compared to WPA3.

WPA3 is designed to be more secure than WPA2, but many devices still support both. By forcing a client to downgrade from WPA3 to WPA2, the attacker can take advantage of WPA2's weaker security.

The goal is to capture a handshake that is easier to crack. WPA2 handshakes, once captured, can be attacked using well-known methods, such as dictionary or brute-force attacks.

The captured handshake is then converted into a format suitable for cracking with hashcat. The downgrade attack makes it possible to obtain this handshake, even if the network is configured to use WPA3, by exploiting the fallback to WPA2.

In [ ]:
#Extract the handshake
sudo hcxpcapngtool -o hashcat.hc22000 capture.pcapng


The command uses hcxpcapngtool to process and extract the handshake data.

The handshake data includes information like the Pairwise Master Key Identifier (PMKID) and EAPOL (Extensible Authentication Protocol over LAN) frames.
These elements are needed for attempting to derive the original pre-shared key (PSK) or password that was used in the handshake process.
This extraction is specific to WPA-EAPOL or WPA-PMKID attacks, as denoted by the hc22000 format. This format is optimized for cracking WPA/WPA2/WPA3 handshakes using hashcat.

In [ ]:
#Crack the handshake
hashcat -m 22000 hashcat.hc22000 -a 3 -w 3 '?d?d?d?d?d?d?d?d'


Hashcat performs a brute-force attack on the extracted handshake data. The goal is to recover the original password (PSK) by trying different combinations until the correct one is found.

**-m 22000**: specifies the attack mode for hashcat, which in this case is for WPA-EAPOL-PMKID combined (used for cracking WPA handshakes).<br>
**hashcat.hc22000**: the input file that contains the extracted handshake data from the previous step.<br>
**-a 3**: sets the attack mode to "mask attack" or brute-force. A mask attack is a more targeted form of brute-force where you define specific character sets or lengths.<Br>
**-w 3**: sets the workload profile, with 3 being a balance between performance and power consumption (suitable for general use).<br>
**'?d?d?d?d?d?d?d?d'**: The mask defines the pattern for the brute-force attack. ?d means any digit (0-9), and repeating it eight times suggests the attack will try all possible combinations of 8-digit numeric passwords.

# Timing Attack

In [ ]:
def elliptic_curve_point_multiplication(secret_scalar, base_point):
    # Simulate varying time based on the value of the scalar
    result = 0
    for bit in bin(secret_scalar)[2:]:  # Process each bit of the secret scalar
        if bit == '1':
            time.sleep(0.0002)  # Simulate longer time for '1' bits
        else:
            time.sleep(0.0001)  # Simulate shorter time for '0' bits
        # Example operation (in reality, much more complex)
        result += int(bit) * base_point
    return result

Elliptic Curve Point Multiplication is a function simulating the elliptic curve point multiplication, which is an operation in elliptic curve cryptography (ECC). In ECC, point multiplication involves multiplying a scalar (a private key or secret) by a point on the elliptic curve (the base point).
The bin(secret_scalar) converts the scalar (secret) into its binary representation. The multiplication process is done bit by bit.

time.sleep(0.0002) simulates the longer time taken to process a '1' bit in the scalar.

time.sleep(0.0001) simulates the shorter time taken to process a '0' bit.

These delays mimic the real-world scenario where operations on certain bits might take longer due to branching or cache behavior.

The line result += int(bit) * base_point is a simplified representation of the elliptic curve operation.

In an actual ECC operation, certain operations might take longer depending on the input data, such as the bits of a cryptographic key. This timing difference is what the attacker would measure and analyze.

In [ ]:
#Measure of execution time
def measure_execution_time(secret_scalar, base_point):
    start_time = time.time()
    elliptic_curve_point_multiplication(secret_scalar, base_point)
    end_time = time.time()
    return end_time - start_time


This function measures the total time it takes to perform the elliptic_curve_point_multiplication operation.

start_time = time.time() records the time before the operation begins.
end_time = time.time() records the time after the operation completes.
The difference between end_time and start_time gives the total execution time, which may vary based on the input (in this case, the scalar value).

In a timing attack, this function would be used to measure how long a cryptographic operation takes, with the goal of detecting patterns that correlate with specific bits or characteristics of the key.

In [ ]:
#Performing the Attack
def timing_attack():
    base_point = 10  # Example base point for elliptic curve
    # Simulate different possible secret scalars (keys)
    possible_secrets = [random.randint(0, 255) for _ in range(5)]  # Small range for example

    timing_results = {}

    for secret_scalar in possible_secrets:
        timings = []
        for _ in range(100):  # Run multiple times to get an average
            execution_time = measure_execution_time(secret_scalar, base_point)
            timings.append(execution_time)

        # Store the average execution time for each secret scalar
        timing_results[secret_scalar] = np.mean(timings)

    # Print the timing results
    for secret, avg_time in timing_results.items():
        print(f"Secret: {secret} | Average Execution Time: {avg_time:.8f} seconds")


The loop for _ in range(100) runs the timing measurement 100 times for each secret. This is done to average out any noise or variability in the timing data, leading to a more reliable measurement.

Averaging helps to filter out random fluctuations and focus on systematic timing differences caused by the secret's bits.

The final loop prints out the average execution time for each possible secret. In a timing attack, the attacker would look for patterns in these timings that correlate with specific bits in the secret key.

The attacker would compare the timing results across different guesses of the secret key. If certain keys consistently take longer or shorter to process, the attacker can use this information to infer the correct key or reduce the key space that needs to be brute-forced.